# 4) Normalization Katmanları — Instance Normalization (InstanceNorm)

Bu defter, **Instance Normalization (IN)** kavramını **CNN (görüntü)** odağında **sıfırdan ileri seviyeye** anlatır.

İçerik:
- IN nedir? Nereden çıkmıştır? (stil transfer bağlamı)
- IN neyi normalize eder? (NCHW eksen mantığı)
- PyTorch: `nn.InstanceNorm2d` / `nn.InstanceNorm1d/3d`
- IN’in BN/GN/LN ile ilişkisi ve farkları
- Object Detection / Segmentation için ne zaman işe yarar, ne zaman zarar verir?
- **BN vs GN vs LN vs IN karşılaştırma tablosu** (tek bakış)

> Not: Gereksiz teori yok. Pratik sezgi + doğru kullanım + doğru kod.


---

## 0) IN için 1 cümle

**InstanceNorm**, her örnek (sample) için **HER KANALI ayrı ayrı** normalize eder:  
Yani her `(N, C)` için, `H×W` üzerinden **mean=0, std=1** yapar.

Bu “kanal başına çok agresif” bir normalizasyondur.


---

## 1) IN nereden çıkmıştır? Kullanılma amacı nedir?

InstanceNorm, özellikle **Neural Style Transfer** ve **image-to-image translation** gibi işlerde popülerleşti.

Neden?
- Stil bilgisinin önemli kısmı: kanal istatistikleri (mean/var)
- IN, her görüntü örneğinde kanal istatistiklerini normalize ederek
  - stili “bastırır”
  - içerik yapısını daha baskın bırakır

Özet:
- **Stil / domain varyasyonunu azaltmak** için güçlü bir araç
- Ama bu güç, bazı görevlerde **bilgi kaybına** neden olabilir.


---

## 2) IN neyi normalize eder? (NCHW eksen mantığı)

Giriş: `x ∈ R^{N×C×H×W}`

IN şu şekilde çalışır:

- Her örnek için (`N` sabit)
- Her kanal için (`C` sabit)
- `H×W` uzaysal alanı üzerinden istatistik alır

Yani normalize edilen set:
- **tek bir kanalın tüm pikselleri**: `(H, W)`

Bu yüzden IN:
- Batch’ten bağımsız ✅
- Kanal başına ayrı normalize ettiği için agresif ⚠️


### 2.1 Matematik (forward)

Bir örnek `n` ve kanal `c` için:

\[
\mu_{n,c} = \frac{1}{HW}\sum_{h,w} x_{n,c,h,w}
\]

\[
\sigma_{n,c}^2 = \frac{1}{HW}\sum_{h,w} (x_{n,c,h,w}-\mu_{n,c})^2
\]

\[
\hat{x}_{n,c,h,w} = \frac{x_{n,c,h,w} - \mu_{n,c}}{\sqrt{\sigma_{n,c}^2+\epsilon}}
\]

Opsiyonel affine:
\[
y = \gamma \hat{x} + \beta
\]


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(42)

---

## 3) PyTorch’ta InstanceNorm

### 3.1 `nn.InstanceNorm2d`
```python
nn.InstanceNorm2d(num_features=C, eps=1e-5, affine=False, track_running_stats=False)
```

Önemli parametreler:
- `affine`: gamma/beta öğrenilsin mi?
- `track_running_stats`: BN gibi running istatistik tutsun mu?
  - IN’de çoğu zaman `False` kullanılır (default).


In [3]:
x = torch.randn(2, 8, 16, 16)  # (N,C,H,W)

in2d = nn.InstanceNorm2d(8, affine=True, track_running_stats=False)
y = in2d(x)

print("x:", x.shape, "y:", y.shape)


x: torch.Size([2, 8, 16, 16]) y: torch.Size([2, 8, 16, 16])


---

## 4) IN gerçekten mean=0, std=1 mi yapıyor?

Evet, **kanal başına** ve **örnek başına**:

- Her (n,c) için `H×W` üzerinden:
  - mean ≈ 0
  - std ≈ 1

Aşağıdaki kontrol, tek bir örnek ve tek kanal için gösterir.


In [4]:
x = torch.randn(1, 3, 8, 8)
in2d = nn.InstanceNorm2d(3, affine=False, track_running_stats=False)
y = in2d(x)

m = y[0, 1].mean().item()
s = y[0, 1].std(unbiased=False).item()
print("Per-(n,c) mean:", m)
print("Per-(n,c) std :", s)


Per-(n,c) mean: 7.450580596923828e-09
Per-(n,c) std : 0.9999959468841553


---

## 5) IN ne zaman iyi, ne zaman tehlikeli?

### İyi olduğu yerler ✅
- **Style transfer** / **image translation**
- Domain farkı çok büyükse (ışık, renk, kamera profili) ve model bu stilden zarar görüyorsa
- Bazı generative modeller (GAN'lar) ve decoder tarafları

### Tehlikeli olduğu yerler ⚠️
- **Object detection**
- **Segmentation** (özellikle sınıf sınırlarında ince detay)
- İnce renk/kontrast ipuçlarının önemli olduğu görevler

Neden tehlikeli?
- IN, her kanalın global (H×W) istatistiğini sıfırlayarak parlaklık/kontrast/renk sinyallerini bastırabilir.


---

## 6) BN, GN, LN, IN karşılaştırma (tek bakış)

Aşağıdaki tablo, “neyin üzerinde normalize edildiğini” gösterir.


| Norm | Normalize edilen set (NCHW) | Batch’e bağımlı mı? | Tipik kullanım | Not |
|---|---|---:|---|---|
| **BatchNorm (BN)** | (N) + H×W, **kanal başına** | ✅ Evet | Classification (büyük batch) | Running stats (train/infer farklı) |
| **GroupNorm (GN)** | (C/G) + H×W, **örnek başına** | ❌ Hayır | Detection/Segmentation (small batch) | GN(1)≈LN, GN(C)≈IN |
| **LayerNorm (LN)** | genelde feature dim; CNN’de çoğu zaman **C** | ❌ Hayır | NLP/Transformer, bazı modern CNN | CNN’de default değil |
| **InstanceNorm (IN)** | H×W, **(n,c) başına** | ❌ Hayır | Style transfer / image translation | Detection’da agresif olabilir |


---

## 7) IN ile GN/LN bağlantısı (sezgi)

GN’nin özel halleri:
- **GN(1)**  → LN benzeri
- **GN(C)**  → IN benzeri

Bu yüzden IN’i “GN’nin uç noktası” gibi düşünebilirsin.


---

## 8) Pratik öneri (CNN / Detection / Segmentation)

- Detection/segmentation için default: **GN** veya **SyncBN**
- Fine-tune: **FrozenBN** sık görülür
- IN: daha çok style/domain dönüşümü tarafı; detection’da dikkatli kullanılmalı.


---

## 9) Özet

- IN: her örnek ve her kanal için `H×W` üzerinden normalize eder.
- Kanal başına mean≈0, std≈1 yapar.
- Style transfer / image translation’da çok kullanışlıdır.
- Detection/segmentation’da bazen faydadan çok zarar verebilir.
- GN, IN ile LN arasında ayarlanabilir bir çözümdür.
